# **Cross-Country Comparison**

---

## **Objective:** 
### Synthesizing the cleaned datasets from Benin, Sierra Leone, and Togo to identify relative solar potential and key differences across countries.

---
---